In [ ]:
%matplotlib widget
from IPython.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))

# `BitArray.draw`: Visualize Multidimensional Bit Arrays

We formulate optimization problems in the mathematical form of a Quadratic Unconstrained Binary Optimization (QUBO) to be able to solve them with Digital Annealer. In the QUBO formulation, we define polynomials and they can have one or multiple dimensions. For example, in the Travelling Salesman Problem (TSP)  $x_{t,c}$ means at time $t$ the traveling salesman is at city $c$ and this problem has 2 dimensions `city` and `time` which can also be represented in a matrix form. TSP bit representation is 2-dimensional. Another example is Vehicle Routing Problem(VRP) where has three dimensions $x_{v, t, l}$ represents a vehicle `v` at time `t` is at location `l`. The VRP problem has 3-dimensions for bit representation. 

This bit representations of QUBOs of these problems are defined using the Digital Annealer Development Kit(dadk) library using `VarShapeSet`. All the variables and their bit array(which bit means what) is defined in `VarShapeSet`. That means for TSP the bit array will have a shape/size of `cities` square(therefore total number of bits needed for this problem are number of cities square) where the axes will be `city` and `time`. While defining the QUBO we can map city and time to the bits easily using VarShapeSet without calculating the index of the bit. In short it helps defining QUBO and problem.

After optimization we can plot bit plots in order to understand and interpret the results. This is made really easy by dadk library. Once we have optimized solution configurations we can use the `draw` method and pass the axes and their order to plot the bit plot. Following is the solution plot example of the TSP. Red bit represents the traveling sales man is visiting that city at that point at that time. The traveling sales man has visited all the cities and its order can be followed from the plot too. Plot axes are also mentioned.

![TSP bit plot](figures/TSP_bit_plot.PNG)

Please also see the VRP example where bit plot shows the round tour for each vehicle. One can follow and see which vehicle is at what location at what time.
![VRP bit plot](figures/VRP_bit_plot.PNG)

# Tutorial

In the following tutorial, you can define number of dimensions between one to five. Aim of this tutorial is to demonstrate user how to use a multidimensional bit array.   

#### Setup scenario

You can specify number of dimensions that will be defined in the problem. Based on the number of dimensions defined those will be named as `A`, `B`, `C`, `D` and `E` respectively. You also need to define the length of each dimension, that means how many elements will be there in each dimension. Finally you can also define how many columns should be there to display the results using the option `columns`. Total number of bits needed for the problem is just multiplication of length of all the dimensions. 

In some problems, it is possible to identify some bits that will never take part in optimization or some can be always be `1` and such bits can be set to `0` or `1` before we send the QUBO for optimization. This will help us to find optimum results faster as we take out many possibilities from the problem and also reduce the number of bits required to represent the problem. This can be done using the argument named `constant_bits` in `VarShapeSet` method. Let us describe it using an example. In VRP all vehicles starts from a depot which is location number `0`. Return to depot is indicated by location `1`. Therefore the vehicle must be there at location `0` at time `0` and nowhere else using `constant_bits` location `0` time `0` is set as `1` and all other bits on location and time axis are set to `0`. Also, on bit plot they appear as light blue in colored when constant_bits are set to `0` otherwise if it is set to `1` then they appear red. It is easy to follow up that light blue colored bits are not taken into consideration in optimization. 
If the bit is selected in optimized solution then it is represented in red in color.

The required bit array from the solution is extracted in a tabular form. The plotting order can also be defined using the argument `order`. For example, if there are dimensions `A`, `B`, `C`  and the order is defined as (`C`, `B`, `A`) the length in dimension `A`, `C` will be plotted on x-axis and `B` will be plotted on y-axis. Constant bits and optimized solution bits can be easily observed in the bit plots.

In [ ]:
import matplotlib.pyplot as plt

from dadk.Optimizer import *

class Demo_BitPlot_Model(OptimizerModel):
    
    ##################################################################################################################

    def __init__(self, persistent_file="Demo_BitPlot.dao"):
        OptimizerModel.__init__(
            self,
            name = 'Demo_BitPlot',
            load_parameter=[
                {'name':'dimensions', 'type':'int_slider', 'value':2, 'min':1, 'max':5, 'label':'dimensions'},
                {'name':'N_A', 'type':'int_slider', 'value':3, 'min':1, 'max':9, 'label':'$N_A$'},
                {'name':'N_B', 'type':'int_slider', 'value':4, 'min':1, 'max':9, 'label':'$N_B$'},
                {'name':'N_C', 'type':'int_slider', 'value':5, 'min':1, 'max':9, 'label':'$N_C$'},
                {'name':'N_D', 'type':'int_slider', 'value':6, 'min':1, 'max':9, 'label':'$N_D$'},
                {'name':'N_E', 'type':'int_slider', 'value':7, 'min':1, 'max':9, 'label':'$N_E$'},
                {'name':'columns', 'type':'int_slider', 'value':2, 'min':1, 'max':9, 'label':'columns'}                
                #{'name':'input_order', 'type':'text', 'value':"ABCDE", 'label':'order like \"ABC\" or \"CAB\"'}                
            ],            
            persistent_file = persistent_file
        )
    
    ##################################################################################################################
    def load(self, dimensions=2, N_A=3, N_B=4, N_C=5, N_D=6, N_E=7, input_order="", columns=2, silent=False):
    ##################################################################################################################

        if dimensions == 1:
            size = (N_A,)
            axis_names=['A']
            constant_bits = np.full(size, -1, np.int8)                    
            constant_bits[N_A//2] = +1
        elif dimensions == 2:
            size = (N_A, N_B)
            axis_names=['A', 'B']
            constant_bits = np.full(size, -1, np.int8)
            for a in range(N_A):
                constant_bits[a, N_B//2] = 0    
            for b in range(N_B):
                constant_bits[N_A//2, b] = 0    
            constant_bits[N_A//2, N_B//2] = +1
        elif dimensions == 3:
            size = (N_A, N_B, N_C)
            axis_names=['A', 'B', 'C']
            constant_bits = np.full(size, -1, np.int8)                    
            for a in range(N_A):
                constant_bits[a, N_B//2, N_C//2] = 0    
            for b in range(N_B):
                constant_bits[N_A//2, b, N_C//2] = 0    
            for c in range(N_C):
                constant_bits[N_A//2, N_B//2, c] = 0    
            constant_bits[N_A//2, N_B//2, N_C//2] = +1
        elif dimensions == 4:
            size = (N_A, N_B, N_C, N_D)
            axis_names=['A', 'B', 'C', 'D']
            constant_bits = np.full(size, -1, np.int8)                    
            constant_bits[N_A//2, N_B//2, N_C//2, N_D//2] = +1
        elif dimensions == 5:
            size = (N_A, N_B, N_C, N_D, N_E)
            axis_names=['A', 'B', 'C', 'D', 'E']
            constant_bits = np.full(size, -1, np.int8)                    
            constant_bits[N_A//2, N_B//2, N_C//2, N_D//2, N_E//2] = +1
        else:
            1/0
            
        order = list(range(dimensions))
        if input_order != "":
            order = []
            for i in range(len(input_order)):
                c = input_order[i].upper()
                if c == 'A':
                    v = 0
                elif c == 'B':
                    v = 1
                elif c == 'C':
                    v = 2
                elif c == 'D':
                    v = 3
                elif c == 'E':
                    v = 4
                else:
                    print("WARNING: unknown %s" % input_order[i])
                    continue
                if v < dimensions: 
                    order.append(v)

        print("axis_names = %s" % str(axis_names))
        print("size = %s" % str(size))
        print("order = %s" % str(order))
        
        if len(order) == dimensions:
            var_shape_set = VarShapeSet(BitArrayShape('x', size, constant_bits=constant_bits, axis_names=axis_names))         
            
            solution = Solution(configuration= np.full(var_shape_set.length, False), var_shape_set=var_shape_set)
            
            solution.draw(name='x', figsize=(6.0, 9.0), max_ax_per_row=columns, order=order)
        else:
            print("ERROR: order has wrong length!")

            
optimizer = Optimizer(Demo_BitPlot_Model(), auto_load=True, show_build=False, show_solve=False, show_track=False) 